In [1]:
import psycopg2
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
conn = psycopg2.connect("dbname=fleebmarket user=fleebmarket host=localhost")


OperationalError: could not connect to server: Connection refused
	Is the server running on host "localhost" (::1) and accepting
	TCP/IP connections on port 5432?
could not connect to server: Connection refused
	Is the server running on host "localhost" (127.0.0.1) and accepting
	TCP/IP connections on port 5432?


In [ ]:
cur = conn.cursor()


In [ ]:
cur.execute("SELECT count(*) as c, author, is_duplicate from search_app_redditadvert where ad_type in ('Selling', 'Sold', 'Buying', 'Purchased') group by author, is_duplicate;")
data = cur.fetchall()

In [ ]:
cur.execute("SELECT author, is_duplicate, created_utc, ad_type from search_app_redditadvert where ad_type in ('Selling', 'Sold', 'Buying', 'Purchased') ;")
data = cur.fetchall()
len(data)

In [ ]:
df = pd.DataFrame(data, columns=["author", "is_duplicate", "created_utc", "ad_type"])
df["created_utc"] = pd.to_datetime(df["created_utc"], utc=True)
df["date"] = df["created_utc"].apply(lambda d: d.date())
df = df[df["date"] > np.datetime64('2021-05-01')]
df

In [ ]:
set(df["ad_type"])

In [ ]:
ax = df.hist(column="created_utc", bins=60, figsize=(20, 5))

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
ad_types = [ "Selling", "Sold", "Buying", "Purchased"]
ax.hist(
    [
        df[df["ad_type"] == ad_type]['created_utc']
        for ad_type in ad_types
    ],
    bins=60,
    stacked=True,
    label=ad_types
)
plt.legend()
plt.savefig("histogram_by_type.png", pad_inches=0, bbox_inches='tight')

In [ ]:
df.groupby('ad_type').count()

In [ ]:
df.groupby('ad_type').count().plot.pie(y="author", autopct="%d", figsize=(8,  8), fontsize=20, ylabel='')
plt.savefig("pie_type.png", pad_inches=0, bbox_inches='tight')

In [ ]:
df_selling = df[df['ad_type'] == "Selling"]

In [ ]:
bins = [1,2,3,5,10,20,30,50,100]
ax = df_selling.groupby('author').size().hist(bins=70, figsize=(30,5), legend=True)
ax.set_xlabel("Number of posts")
ax.set_ylabel("Number of authors")
plt.savefig("Number of author by number of adverts.png", pad_inches=0, bbox_inches='tight')

In [ ]:
from collections import defaultdict
hist_data_raw = defaultdict(lambda: 0)
for value in df_selling.groupby('author').size():
    hist_data_raw[value] += 1
hist_data_raw

In [ ]:
hist_data = {}
for key, value in hist_data_raw.items():
    hist_data[key] = value * key
hist_data

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
ax.bar(x=list(hist_data.keys()), height=list(hist_data.values()), width=1)
ax.set_xlabel("Number of authors")
ax.set_ylabel("Cummulative number of posts")

In [ ]:
def is_last_post(row):
    author = row['author']
    df_author = df[df['author'] == author]
    return all((row['created_utc'] >= created) for created in df_author['created_utc'])
df_selling['is_last_post'] = df_selling.apply(is_last_post, axis="columns")
df_selling

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
ax.hist(
    [
        df_selling[df_selling["is_last_post"] == val]['created_utc']
        for val in [True, False]
    ],
    bins=30,
    stacked=True,
    label=["Last post", "Not last post"]
)
plt.legend()
plt.savefig("last post.png", pad_inches=0, bbox_inches='tight')

In [ ]:
fig, ax = plt.subplots(figsize=(20,5))
ax.hist(
    [
        df_selling[(df_selling["is_last_post"] == val1) & (df_selling["is_duplicate"] == val2)]['created_utc']
        for val1, val2 in ((True, False),(False, True),(False, False))
    ],
    bins=60,
    stacked=True,
    label=["Last post", "Duplicate", "Not duplicate, not last post"] 
)
plt.legend()
plt.savefig("duplicates.png", pad_inches=0, bbox_inches='tight')

In [ ]:
df_selling[df_selling['is_duplicate'] & df_selling['is_last_post']]

In [ ]:
df_selling.groupby(['is_duplicate', 'is_last_post'], as_index=False).count()

In [ ]:
df_selling["cat"] = df_selling.apply(
    lambda row: "Duplicate" if row["is_duplicate"] else ("Last post" if row["is_last_post"] else "Not duplicate, not last post"),
    axis="columns"
)

In [ ]:
df_selling.groupby("cat").count().plot.pie(y="author", autopct="%d", figsize=(8,  8), fontsize=20, ylabel='')

In [ ]:
fig, ax = plt.subplots(figsize=(30, 5))
ax.hist(df_selling.groupby('author').size(), bins=70, range=(1,70), linewidth=1.2, edgecolor='black', label="all posts")
ax.hist(df_selling[~df_selling["is_duplicate"]].groupby('author').size(), bins=70, range=(1,70), edgecolor='black', label="without duplicates")

plt.legend()
plt.yscale("log")
ax.set_xlabel("Number of posts")
ax.set_ylabel("Number of authors")

plt.title("Number of author by number of posts")
plt.savefig("Number of author by number of posts - with non-duplicates (log scale).png", pad_inches=0, bbox_inches='tight')